[Report Google Doc](https://docs.google.com/document/d/1CWJNf2I3JUKnVm02NwIsNZxq3Y5GMNvZSBCoo-qSc7M/edit?ts=5bcb4b11)

# What does a county's chemical contaminants say about its industrial activity?

## We need a profile of each county's industrial activity.
We will build a profile of each county for a given year by concatenating a vector representing a county's water usage in 2010 and their occupation statistics for the given year.
We normalize the water usage data for population and droughts during 2010 by  predicting water usage based on population and drought, and subtracting these predictions from the water usage data. This will give us a baseline measurement of a county's industrial activity in 2010.
We will also need to remove socioeconomic patterns from the industrial occupation data. We do this by performing PCA on water usage+occupation+earnings+education data and remove the eigenvectors heavily corresponding to earnings+education patterns.
We will then concatenate the water usage data and the occupation data into a single feature vector for each year representing the county's industrial profile for that year. We normalize every column to a mean and variance of 1.
## We can then correlate the profile with chemical contaminant levels
We will build a regression model to measure what an increase/decrease in contaminants say about industrial activity and vise versa. We will then cluster chemicals based on how they relate to industrial profiles (similarity of regression coefficients).
We will then build a regression model from contaminants to changes in industrial activity (instead of the value at a given year). We then perform the same covariance and clustering as before.

## Statistics:
### Step 1
- Build regression model from historical education attainment and the 2010 population to water usage. Subtract this inferred value from water usage.
- Calculate covariance between occupation and water, earnings, and education.
- Perform PCA on water usage, occupation, earnings and education (normalizing first).
- Recompute occupation data with SES-heavy components removed.
- Recalculate covariance between occupation and water, earnings, and education.
- Concatenate occupation vector and water usage data (duplicating water usage data for each year).

### Step 2
- Build a regression model from industrial vector to chemical contaminants data (normalizing first).
- Calculate contaminant predictions after perturbing regression and vise-versa.
- Cluster chemicals based on their regression coefficients.
- Repeat except using year-wise change in industrial vector (y_n+1 - y_n) instead of y_n.

## Visualizations/graphs:
- Covariance matrix of water usage, drought and population before and after normalizing water usage data.
- Covariance matrix of occupation and peripherals before and after PCA.
- Visualize PCA removal of SES.
- Feature importances of the industrial vector.
- Visualize effect of perturbing regression?
- Cluster map visualizing similarity of regression coefficients for each chemical.

# Step 1: Build industrial feature vector

In [0]:
import plotly 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

water_df = pd.read_csv("water_usage.csv")
water_dict_df = pd.read_csv("water_usage_dictionary.csv")
industry_df = pd.read_csv("industry_occupation.csv", encoding = "latin1")
education_df = pd.read_csv("education_attainment.csv", encoding = "latin1")
earnings_df = pd.read_csv("earnings.csv", encoding = "latin1")
chemicals_df = pd.read_csv("chemicals.csv", encoding = "latin1")
droughts_df = pd.read_csv("droughts.csv", encoding = "latin1")

In [0]:
new_droughts_df = droughts_df.copy()

def normalize(labels_df):
  global new_droughts_df, water_df
  droughts_df = new_droughts_df
  """ Normalize any labels_df with fips for 2010 using drought and population """
  droughts_df['valid_start'] = pd.to_datetime(droughts_df['valid_start'])
  droughts_df['valid_end'] = pd.to_datetime(droughts_df['valid_end'])

  droughts_df = droughts_df[droughts_df['valid_end'].dt.year >= 2010]
  droughts_df = droughts_df[droughts_df['valid_start'].dt.year <= 2010]
  droughts_df = droughts_df[["fips", "none", "d0", "d1", "d2", "d3", "d4"]]

  features_df = water_df[["fips", "population"]].merge(droughts_df, on="fips").fillna(0)

  features_df = features_df.merge(labels_df.fillna(0), on="fips")

  fips = [x for x in features_df["fips"]]
  features = [x[1].tolist() for x in features_df[["none", "d0", "d1", "d2", "d3", "d4", "population"]].iterrows()]
  labels = [x[1].tolist() for x in features_df.drop(columns=["fips", "none", "d0", "d1", "d2", "d3", "d4", "population"]).iterrows()]

  water_normalizer = linear_model.LinearRegression()
  water_normalizer.fit(features, labels)
  labels = np.subtract(np.array(labels), np.array(water_normalizer.predict(features)))

  return {k: v for k, v in zip(fips, labels)}

In [0]:
M = {}

# intersec_fips
intersec_fips = list(set(water_df.fips).intersection(set(industry_df.fips)))

# drop invariant cols
drop_industry_df = industry_df.drop(['geo_id','county'], axis=1).dropna(0)
drop_water_dict = normalize(water_df.drop(['state','state_fips','county','county_fips','year','population'], axis=1))

for fips in intersec_fips:
    ind_vecs = drop_industry_df[:][drop_industry_df.fips == fips]
    ind_years = ind_vecs.year.values
    #print(ind_years)
    ind_vecs = ind_vecs.drop(['year','fips'],axis = 1)
    ind_vecs = ind_vecs.astype(float).divide(ind_vecs["total_employed"].astype(float), axis=0)
    ind_vecs = ind_vecs.values
    #print(ind_vecs)
    wat_vec = drop_water_dict[fips]
    wat_vec = [0 if np.isnan(v) else v for v in wat_vec]
    for idx, vec in enumerate(ind_vecs):
        vec_nan = [0 if np.isnan(v) else v for v in vec]
        M[str(fips) + str(ind_years[idx])] = wat_vec + vec_nan
col_names = list(drop_water_df.columns.values)[1:]+list(industry_df.columns.values[2:-2])
col_name_dict = {name: idx for idx, name in enumerate(col_names)}
county_dict = M



# Step 2: Correlating industrial activity and contaminant levels.

In [0]:
''' Import necessary libraries '''
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

In [0]:
""" Generate the contamination dataset """

changed_chemicals_df = chemicals_df.copy()
changed_chemicals_df["fips"] = changed_chemicals_df["fips"].astype(str) + changed_chemicals_df["year"].astype(str)
# Grab dataset from dataframe
contamination_dict = {}
for _, row in changed_chemicals_df.iterrows():
  if row["fips"] not in contamination_dict:
    contamination_dict[row["fips"]] = {}
  contamination_dict[row["fips"]][row["chemical_species"] + "(" + row["unit_measurement"] + ")"] = row["value"]

# Build a map from feature idx to chemical
chemical_index_map = []
for fips, value in contamination_dict.items():
  for chem, _ in value.items():
    if chem not in chemical_index_map:
      chemical_index_map.append(chem)

# Build an index of overall idx to fips
idx_to_fips = []
contamination_vectors = []
for fips in contamination_dict.keys():
  idx_to_fips.append(fips)
  feature_vec = [0 for _ in range(len(chemical_index_map))]
  for idx, chem in enumerate(chemical_index_map):
    if chem not in contamination_dict[fips]:
      feature_vec[idx] = 0
    else:
      feature_vec[idx] = contamination_dict[fips][chem]
  contamination_vectors.append(feature_vec)


# Normalize contamination vectors to mean, var of 1.
# We store the reverse scaler in reverse_contaminant_normalization(x).
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(contamination_vectors)
reverse_contaminant_normalization = lambda x: x * scaler.var_ + scaler.mean_
contamination_vectors = scaler.transform(contamination_vectors)

In [0]:
""" Clean up industrial activity dataset for our uses """

county_data = [(key, v) for key, v in county_dict.items()]

# Drop contaminants we don't have county vecs for.
county_fips = [x[0] for x in county_data]
new_idx_to_fips = [x for x in idx_to_fips if x in county_fips]
new_contamination_vectors = [contamination_vectors[idx_to_fips.index(x)] for x in new_idx_to_fips]
idx_to_fips = new_idx_to_fips
contamination_vectors = new_contamination_vectors

# Build county vectors
county_fips = {x[0]: x[1] for x in county_data}
county_vectors = []
for fips in idx_to_fips:
  county_vectors.append(county_fips[fips])

scala = StandardScaler(with_mean=True, with_std=True)
scala.fit(county_vectors)
county_vectors = scala.transform(county_vectors)

In [0]:
"""
Given a dataset of feature vectors chemical contaminant levels, try to predict
the vector representing the industrial activity for a given county at a given
year. Cluster the chemicals based on the similarity of their correlations to
the industrial activity.
"""

chem_to_county = linear_model.LinearRegression()
chem_to_county.fit(contamination_vectors, county_vectors)
coefs = chem_to_county.coef_
    
kmeans = KMeans(n_clusters=2, random_state=0).fit(coefs)
cluster_assignments = {}
coefffs = {}
for i, chem_name in contamination_idx_to_chemical.items():
  cluster_assignments[chem_name] = kmeans.labels_[i]
  coefffs[chem_name] = sklearn.manifold.TSNE().fit_transform(coefs[i])
print(cluster_assignments)
print(coefffs)

In [0]:
"""
Given a dataset of feature vectors representing the industrial activity
for a given county at a given year, try to predict the chmemical contaminant
level. What happens to contaminant levels when we perturb the county vectors?
"""

from sklearn import linear_model

def run(cols):
  global county_vectors, contamination_vectors, county_to_chem

  # PUT DELTA VEC HERE:
  zero_vec = [0 for _ in range(len(county_vectors[0]))]
  delta_vec = [0 for _ in range(len(county_vectors[0]))]
  for n in [*cols]:
    delta_vec[col_name_dict[n]] = 1
  delta_vec = np.nan_to_num(delta_vec / scala.var_)

  county_to_chem = linear_model.Ridge(alpha = .5)
  county_to_chem.fit(county_vectors, contamination_vectors)

  results = county_to_chem.predict([delta_vec])[0] - county_to_chem.predict([zero_vec])[0]
  effect = {contamination_idx_to_chemical[i]: value
            for i, value in enumerate(results * scaler.var_)}
  print(effect)
  return effect

In [0]:
# cols 
ind_cols = col_names[20:29]
irr_cols = col_names[29:36]
mining_cols = col_names[62:71]
livestock_cols = col_names[50:53]

In [0]:
import numpy as np
import matplotlib.pyplot as plt


run(ind_cols)
run(irr_cols)
run(mining_cols)
run(livestock_cols)

data = [[ 66386, 174296,  75131, 577908,  32015],
        [ 58230, 381139,  78045,  99308, 160454],
        [ 89135,  80552, 152558, 497981, 603535],
        [ 78415,  81858, 150656, 193263,  69638],
        [139361, 331509, 343164, 781380,  52269]]

columns = ("Industrial", "Irrigation", "Mining", "Livestock")
rows =  run(irr_cols).keys()

values = np.arange(0, 2500, 500)
value_increment = 1000

# Get some pastel shades for the colors
colors = plt.cm.BuPu(np.linspace(0, 0.5, len(rows)))
n_rows = len(data)

index = np.arange(len(columns)) + 0.3
bar_width = 0.4

# Initialize the vertical-offset for the stacked bar chart.
y_offset = np.zeros(len(columns))

# Plot bars and create text labels for the table
cell_text = []
for row in range(n_rows):
    plt.bar(index, data[row], bar_width, bottom=y_offset, color=colors[row])
    y_offset = y_offset + data[row]
    cell_text.append(['%1.1f' % (x / 1000.0) for x in y_offset])
# Reverse colors and text labels to display the last value at the top.
colors = colors[::-1]
cell_text.reverse()

# Add a table at the bottom of the axes
the_table = plt.table(cellText=cell_text,
                      rowLabels=rows,
                      rowColours=colors,
                      colLabels=columns,
                      loc='bottom')

# Adjust layout to make room for the table:
plt.subplots_adjust(left=0.2, bottom=0.2)

plt.ylabel("Loss in ${0}'s".format(value_increment))
plt.yticks(values * value_increment, ['%d' % val for val in values])
plt.xticks([])
plt.title('Loss by Disaster')

plt.show()

In [0]:
from sklearn.decomposition import PCA

In [0]:

X = columns 
pca = PCA()
pca.fit(X)


In [0]:
import copy

def pca_normalize(county_dict):
  earnings_df = pd.read_csv("earnings.csv", encoding = "latin1")
  earnings_df["fips"] = earnings_df["fips"].astype(str) + earnings_df["year"].astype(str)
  earnings_df.drop(["pub_admin", "year", "county", "geo_id"], axis=1)
  x = copy.deepcopy(county_dict)
  feat_n = None

  for row in earnings_df.iterrows():
    print(x)
    feat_n = len(list(row)[1:])
    x[row["fips"]] += list(row)[1:]
    
  keys = x.keys()
  data = [x[key] for key in keys]
  pca = PCA()
  pca.fit(data)
  pca.components_.pop(IND)
  pca.singular_values_.pop(IND)
  y = sum([x * y for x, y in zip(pca.components_, pca.singular_values_)])
  y = y[:-feat_n]
  

new_county_dict = None

In [0]:
list(next(earnings_df.iterrows())[1])

In [0]:
array = np.array

A = {'Uranium(micrograms/L)': array([-0.00584229,  0.72542275,  0.08034749,  0.02117824, -0.03213056,
      -0.04028493]), 'Arsenic(micrograms/L)': array([ 0.0008375 , -0.02308488,  0.07815932,  0.04418895, -0.03681452,
       0.06958854]), 'DEHP(micrograms/L)': array([-0.02464186, -0.12505089, -0.09959427, -0.06536517,  0.02902133,
       0.08917486]), 'Nitrates(milligrams/L)': array([ 9.02643105e-04,  1.15423081e+00,  9.82211576e-02,  2.61018523e-02,
      -6.28437209e-02, -5.42133463e-02]), 'Halo-Acetic Acid(micrograms/L)': array([ 0.00754173, -0.39016013,  0.08838049, -0.03296108, -0.07166619,
       0.03815253]), 'Trihalomethane(micrograms/L)': array([ 0.00165637,  1.0730999 ,  0.10397881,  0.02175341, -0.06810205,
      -0.04833789])}

In [0]:
import sklearn
keys = A.keys()
v = [A[k] for k in keys]

x = sklearn.manifold.TSNE().fit_transform(v)

for key, v in zip(keys, x):
  print(key, v/100)
